In [1]:
import os

import numpy as np

import torch
import torch.nn as nn

from torch.autograd import Variable

import torchvision.utils

from new_data_loader import get_loader
from make_gif import make_gif

from DiscrimiatorModule import Discriminator
from GeneratorModule import Generator

In [2]:
def init_weights(m) :
    name = type(m)

    if name == nn.Conv3d or name == nn.ConvTranspose2d or name == nn.ConvTranspose3d :
        m.weight.data.normal_(0.0, 0.01)
        m.bias.data.fill_(0)
    elif name == nn.BatchNorm2d or name == nn.BatchNorm3d :
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

In [ ]:
pre_train = True

batch_size = 64
video_size = 64
epoch_size = 1000
        
#check GPU
is_gpu = torch.cuda.is_available()
print(is_gpu)

if is_gpu :
    dtype = torch.cuda.FloatTensor
else :
    dtype = torch.FloatTensor

if pre_train :
    D = torch.load('D.ckpt').type(dtype)
    G = torch.load('G.ckpt').type(dtype)
else :
    D = Discriminator()
    D = D.type(dtype)

    G = Generator()
    G = G.type(dtype)

    D.apply(init_weights)
    G.apply(init_weights)

criterion = nn.BCEWithLogitsLoss().type(dtype)

d_optimizer = torch.optim.Adam(D.parameters(), lr=2e-4, betas=(0.5, 0.999))
g_optimizer = torch.optim.Adam(G.parameters(), lr=2e-4, betas=(0.5, 0.999))

True


In [ ]:
data_loader = get_loader(data_path='./dataset', image_size=video_size, batch_size=batch_size, num_workers=2)

for epoch in range(384, epoch_size + 1) :
    for iter, (video, y) in enumerate(data_loader) :
        local_batch_size = video.size()[0]
        
        real_labels = Variable(torch.ones(local_batch_size, 2).type(dtype))
        fake_labels = Variable(torch.zeros(local_batch_size, 2).type(dtype))
        
        # 1. Train Discriminator
        video_data = Variable(video).type(dtype)
        y = Variable(y).type(dtype)
        
        y_data = torch.unsqueeze(torch.unsqueeze(torch.unsqueeze(y, -1), -1), -1)
        y_data = y_data.expand(local_batch_size, 6, 32, 64, 64)
       
        
        
        # 1-1. Real Video
        outputs = D(video_data, y_data).view(local_batch_size, 2)
        d_loss_real = criterion(outputs, real_labels)

        
        
        # 1-2. Fake Video
        z = Variable(torch.randn(local_batch_size, 100) * 0.01).type(dtype)
        fake_videos = G(z, y)
        outputs = D(fake_videos, y_data).view(local_batch_size, 2)
        d_loss_fake = criterion(outputs, fake_labels)

        d_loss = d_loss_real + d_loss_fake
        
        D.zero_grad()
        G.zero_grad()
        d_loss.backward()
        d_optimizer.step()





        # 2. Train Generator
        z = Variable(torch.randn(local_batch_size, 100) * 0.01).type(dtype)
        fake_videos = G(z, y)
        outputs = D(fake_videos, y_data).view(local_batch_size, 2)

        g_loss = criterion(outputs, real_labels)
        D.zero_grad()
        G.zero_grad()

        g_loss.backward()
        g_optimizer.step()
          
        print('Epoch [%d/%d], Iter [%d/%d], d_loss: %.4f, g_loss: %.4f' % (epoch, epoch_size, iter, len(data_loader), d_loss.data[0], g_loss.data[0]))
    
    print('Model saving...')
    
    torch.save(D, 'D.ckpt')
    torch.save(G, 'G.ckpt')

Epoch [384/1000], Iter [0/49], d_loss: 0.2224, g_loss: 2.0415
Epoch [384/1000], Iter [1/49], d_loss: 0.1755, g_loss: 6.3966
Epoch [384/1000], Iter [2/49], d_loss: 0.0084, g_loss: 9.1431
Epoch [384/1000], Iter [3/49], d_loss: 0.0066, g_loss: 8.1735
Epoch [384/1000], Iter [4/49], d_loss: 0.0121, g_loss: 7.2475
Epoch [384/1000], Iter [5/49], d_loss: 0.0454, g_loss: 8.9441
Epoch [384/1000], Iter [6/49], d_loss: 0.0042, g_loss: 8.0692
Epoch [384/1000], Iter [7/49], d_loss: 0.0257, g_loss: 8.9309
Epoch [384/1000], Iter [8/49], d_loss: 0.0479, g_loss: 6.7944
Epoch [384/1000], Iter [9/49], d_loss: 0.0291, g_loss: 6.1598
Epoch [384/1000], Iter [10/49], d_loss: 0.0217, g_loss: 8.0372
Epoch [384/1000], Iter [11/49], d_loss: 0.0011, g_loss: 8.1110
Epoch [384/1000], Iter [12/49], d_loss: 0.0019, g_loss: 6.4925
Epoch [384/1000], Iter [13/49], d_loss: 0.0156, g_loss: 6.9862
Epoch [384/1000], Iter [14/49], d_loss: 0.0098, g_loss: 5.9769
Epoch [384/1000], Iter [15/49], d_loss: 0.0238, g_loss: 7.0272
Ep

Epoch [386/1000], Iter [32/49], d_loss: 0.0336, g_loss: 4.0108
Epoch [386/1000], Iter [33/49], d_loss: 0.1033, g_loss: 6.9979
Epoch [386/1000], Iter [34/49], d_loss: 0.3524, g_loss: 3.1179
Epoch [386/1000], Iter [35/49], d_loss: 0.0522, g_loss: 2.0328
Epoch [386/1000], Iter [36/49], d_loss: 0.3211, g_loss: 10.7161
Epoch [386/1000], Iter [37/49], d_loss: 0.0633, g_loss: 11.8938
Epoch [386/1000], Iter [38/49], d_loss: 0.0912, g_loss: 12.0440
Epoch [386/1000], Iter [39/49], d_loss: 0.0421, g_loss: 10.5297
Epoch [386/1000], Iter [40/49], d_loss: 0.0354, g_loss: 9.8640
Epoch [386/1000], Iter [41/49], d_loss: 0.0164, g_loss: 7.4268
Epoch [386/1000], Iter [42/49], d_loss: 0.0023, g_loss: 6.8865
Epoch [386/1000], Iter [43/49], d_loss: 0.0054, g_loss: 7.0230
Epoch [386/1000], Iter [44/49], d_loss: 0.0601, g_loss: 7.3177
Epoch [386/1000], Iter [45/49], d_loss: 0.0022, g_loss: 8.8232
Epoch [386/1000], Iter [46/49], d_loss: 0.0120, g_loss: 7.1406
Epoch [386/1000], Iter [47/49], d_loss: 0.1467, g_l

Epoch [389/1000], Iter [15/49], d_loss: 2.9704, g_loss: 4.4706
Epoch [389/1000], Iter [16/49], d_loss: 0.9614, g_loss: 15.0545
Epoch [389/1000], Iter [17/49], d_loss: 0.4481, g_loss: 10.9976
Epoch [389/1000], Iter [18/49], d_loss: 0.0850, g_loss: 8.3482
Epoch [389/1000], Iter [19/49], d_loss: 0.0639, g_loss: 8.9187
Epoch [389/1000], Iter [20/49], d_loss: 0.1109, g_loss: 6.6247
Epoch [389/1000], Iter [21/49], d_loss: 0.2272, g_loss: 11.2364
Epoch [389/1000], Iter [22/49], d_loss: 0.0772, g_loss: 9.7366
Epoch [389/1000], Iter [23/49], d_loss: 0.1237, g_loss: 7.9761
Epoch [389/1000], Iter [24/49], d_loss: 0.0521, g_loss: 5.6863
Epoch [389/1000], Iter [25/49], d_loss: 0.0455, g_loss: 6.4406
Epoch [389/1000], Iter [26/49], d_loss: 0.0237, g_loss: 5.2376
Epoch [389/1000], Iter [27/49], d_loss: 0.1569, g_loss: 8.9750
Epoch [389/1000], Iter [28/49], d_loss: 0.0571, g_loss: 8.3331
Epoch [389/1000], Iter [29/49], d_loss: 0.0622, g_loss: 7.9517
Epoch [389/1000], Iter [30/49], d_loss: 0.0296, g_lo